In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

%matplotlib inline

from sklearn.metrics import mean_squared_error

from qiskit import *
from qiskit.algorithms import IterativeAmplitudeEstimation, EstimationProblem
from qiskit.circuit.library import LinearAmplitudeFunction
from qiskit_aer.primitives import Sampler
from qiskit_finance.circuit.library import LogNormalDistribution
from qiskit_finance.applications.estimation import EuropeanCallPricing

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2023-06-04 03:58:39,246: Credentials are already in use. The existing account in the session will be replaced.


We first want to define a function to randomly sample the estimates we get from the quantum algorithm.

In [7]:
# Outputs an array containing the exact value, the estimated value (from the quantum estimation),
# and the lower and upper bound of the corresponding confidence interval.
def GetPricesRandom():
    
    S = np.round(random.uniform(1, 9999), decimals=4)  # initial spot price
    vol = np.round(random.uniform(0.1, 1), decimals=4) # volatility
    r = 0.05                                           # annual interest rate
    T = (random.uniform(1, 365))/365                   # days to maturity
    num_uncertainty_qubits = 3                         # number of qubits to represent the uncertainty
    c_approx = 0.25                                    # approximation scaling for the payoff function


    # resulting parameters for log-normal distribution
    mu = (r - 0.5 * vol**2) * T + np.log(S)
    sigma = vol * np.sqrt(T)
    mean = np.exp(mu + sigma**2 / 2)
    variance = (np.exp(sigma**2) - 1) * np.exp(2 * mu + sigma**2)
    stddev = np.sqrt(variance)

    # lowest and highest value considered for the spot price; in between, an equidistant discretization is considered.
    low = np.maximum(0, mean - 3 * stddev)
    high = mean + 3 * stddev
    strike_price = np.round(random.uniform(low, high), decimals=4)

    # construct A operator for QAE for the payoff function by
    # composing the uncertainty model and the objective
    uncertainty_model = LogNormalDistribution(
        num_uncertainty_qubits, mu=mu, sigma=sigma**2, bounds=(low, high)
    )
    
    # setup piecewise linear objective fcuntion
    breakpoints = [low, strike_price]
    slopes = [0, 1]
    offsets = [0, 0]
    f_min = 0
    f_max = high - strike_price
    european_call_objective = LinearAmplitudeFunction(
        num_uncertainty_qubits,
        slopes,
        offsets,
        domain=(low, high),
        image=(f_min, f_max),
        breakpoints=breakpoints,
        rescaling_factor=c_approx,
    )

    # construct A operator for QAE for the payoff function by
    # composing the uncertainty model and the objective
    num_qubits = european_call_objective.num_qubits
    european_call = QuantumCircuit(num_qubits)
    european_call.append(uncertainty_model, range(num_uncertainty_qubits))
    european_call.append(european_call_objective, range(num_qubits))
    
    # plot exact payoff function (evaluated on the grid of the uncertainty model)
    x = uncertainty_model.values
    y = np.maximum(0, x - strike_price)
    
    # evaluate exact expected value (normalized to the [0, 1] interval)
    exact_value = np.dot(uncertainty_model.probabilities, y)
    
    # set target precision and confidence level
    epsilon = 0.01
    alpha = 0.05

    problem = EstimationProblem(
        state_preparation=european_call,
        objective_qubits=[3],
        post_processing=european_call_objective.post_processing,
    )
    # construct amplitude estimation
    ae = IterativeAmplitudeEstimation(
        epsilon_target=epsilon, alpha=alpha, sampler=Sampler(run_options={"shots": 100})
    )
    result = ae.estimate(problem)
    conf_int = np.round(np.array(result.confidence_interval_processed), decimals=4)
    
    # return the exact value, estimate, and corresponding confidence interval
    return [np.round(exact_value, decimals=4), np.round(result.estimation_processed, decimals=4), conf_int[0], conf_int[1]]


Next we run our function to get 500 random samples.

In [ ]:
# Randomly sample salues generated from estimation algorithm
fpe = []  # array of fair price estimates
fp = []   # array of true fair prices

# Sample 500 random points
for i in range(500):
    values = GetPricesRandom()
    fpe.append(values[1])
    fp.append(values[0])

In [ ]:
# Plot a straight line
x = np.linspace(0, max(fpe), 100)
y = x
plt.plot(x, y, '-r')

# Plot Values
plt.scatter(fpe, fp)
plt.title("Estimated vs True Fair Prices")
plt.xlabel("Estimated Fair Price")
plt.ylabel("True Fair Price")

# Get MSE and RMSE
mse = mean_squared_error(fp, fpe)
rmse = np.sqrt(mse)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)

Note that our plotted data closely follows a straight line, a sign that the estimates are close to the analytically-derived results.

In [ ]:
import qiskit.tools.jupyter
%qiskit_copyright